In [ ]:
#!/usr/bin/env python
"""
Ways to evaluate partition function
Z_eval = 'Lap'         : Laplace approximation (default)
         'Lap+Sam[+P]' : Laplace approximation + importance sampling
         'GLap[+P]'    : generalized Laplace approximation
         'GLap+Sam[+P]': generalized Laplace approximation + importance sampling
         'Lap+Fey'     : Laplace approximation + Feynman diagrams

Methods of posterior sampling
pt_method =  None       : no sampling will be performed (default)
            'Lap[+P]'   : sampling from Laplace approximation + importance weight
            'GLap[+P]'  : sampling from generalized Laplace approximation + importance weight
            'MMC'       : sampling using Metropolis Monte Carlo

Note: [+P] means this task can be done in parallel
"""
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
TINY_FLOAT64 = sp.finfo(sp.float64).tiny

execfile('test_header.py')

num_trials = 100
num_powers = 6

data_seeds = np.random.randint(0, 10**8, num_trials)
num_pt_samples = 10**num_powers

w_mean = np.zeros([num_trials,num_powers])
w_mean_std = np.zeros([num_trials,num_powers])
H_mean = np.zeros([num_trials,num_powers])
H_mean_std = np.zeros([num_trials,num_powers])

for i in range(num_trials):
    
    print ''
    print '#', i
    data_seed = data_seeds[i]
    print 'data_seed =', data_seed
    
    results = TestCase(N=50,
                       data_seed=data_seed,
                       G=100,
                       alpha=3,
                       bbox=[-20,20],
                       Z_eval='GLap',
                       num_Z_samples=0,
                       DT_MAX=1.0,
                       pt_method='GLap',
                       num_pt_samples=num_pt_samples,
                       fix_t_at_t_star=True
                       ).run()

    xs = results.results.bin_centers
    Q_true = results.Q_true
    Q_star = results.results.Q_star
    Q_samples = results.results.Q_samples
    sample_weights = results.results.phi_weights

    # Compute entropy of Q_true and Q_samples
    h = xs[1] - xs[0]
    H_true = -sp.sum(Q_true * sp.log2(Q_true + TINY_FLOAT64)) * h
    H_samples = np.zeros(num_pt_samples)
    for k in range(num_pt_samples):
        Q_k = Q_samples[:,k]
        H_samples[k] = -sp.sum(Q_k * sp.log2(Q_k + TINY_FLOAT64)) * h

    for p in range(num_powers):
        num_samples = 10**(p+1)
        weights = sample_weights[0:num_samples]
        entropies = H_samples[0:num_samples]
        # Compute sample mean of the weights and its std
        w_sample_mean = sp.mean(weights)
        w_sample_mean_std = sp.std(weights) / sp.sqrt(num_samples)
        w_mean[i,p] = w_sample_mean
        w_mean_std[i,p] = w_sample_mean_std
        # Compute sample mean of the entropies and its std
        H_sample_mean = sp.sum(entropies * weights) / sp.sum(weights)
        H_sample_std = sp.sqrt(sp.sum(entropies**2 * weights) / sp.sum(weights) - H_sample_mean**2)
        H_sample_mean_std = H_sample_std / sp.sqrt(num_samples)
        H_mean[i,p] = H_sample_mean
        H_mean_std[i,p] = H_sample_mean_std

# Plot 1: w mean vs log N
plt.figure(1)
x = range(num_powers) + np.ones(num_powers)
for i in range(num_trials):
    plt.scatter(x, w_mean[i,:], color='black')
plt.xlabel('log N')
plt.ylabel('w mean')
plt.grid(axis='y', linestyle='--', color='grey')
plt.savefig('w_mean')

# Plot 2: w mean std vs log N
plt.figure(2)
x = range(num_powers) + np.ones(num_powers)
for i in range(num_trials):
    plt.scatter(x, w_mean_std[i,:], color='black')
plt.xlabel('log N')
plt.ylabel('w mean std')
plt.grid(axis='y', linestyle='--', color='grey')
plt.savefig('w_mean_std')

# Plot 3: H mean vs log N
plt.figure(3)
x = range(num_powers) + np.ones(num_powers)
for i in range(num_trials):
    plt.scatter(x, H_mean[i,:], color='black')
plt.xlabel('log N')
plt.ylabel('H mean (bits)')
plt.grid(axis='y', linestyle='--', color='grey')
plt.savefig('H_mean')

# Plot 4: H mean std vs log N
plt.figure(4)
x = range(num_powers) + np.ones(num_powers)
for i in range(num_trials):
    plt.scatter(x, H_mean_std[i,:], color='black')
plt.xlabel('log N')
plt.ylabel('H mean std (bits)')
plt.grid(axis='y', linestyle='--', color='grey')
plt.savefig('H_mean_std')

print ''
print '--- done ---'
print

In [ ]:
plt.show()

In [ ]:
indices = []
for i in range(num_trials):
    if np.all(w_mean[i,:] < 5) and np.all(w_mean_std[i,:] < 5):
        indices.append(i)
        
sub_num_trials = len(indices)
print sub_num_trials

w_mean_sub = np.zeros([sub_num_trials, num_powers])
w_mean_std_sub = np.zeros([sub_num_trials, num_powers])
H_mean_sub = np.zeros([sub_num_trials, num_powers])
H_mean_std_sub = np.zeros([sub_num_trials, num_powers])
for i in range(sub_num_trials):
    w_mean_sub[i,:] = w_mean[indices[i],:]
    w_mean_std_sub[i,:] = w_mean_std[indices[i],:]
    H_mean_sub[i,:] = H_mean[indices[i],:]
    H_mean_std_sub[i,:] = H_mean_std[indices[i],:]

In [ ]:
# Plot sub data

# Plot 1: w mean vs log N
plt.figure(21)
x = range(num_powers) + np.ones(num_powers)
for i in range(sub_num_trials):
    plt.scatter(x, w_mean_sub[i,:], color='black')
plt.xlabel('log N')
plt.ylabel('w mean')
plt.grid(axis='y', linestyle='--', color='grey')
plt.savefig('w_mean_sub')

# Plot 2: w mean std vs log N
plt.figure(22)
x = range(num_powers) + np.ones(num_powers)
for i in range(sub_num_trials):
    plt.scatter(x, w_mean_std_sub[i,:], color='black')
plt.xlabel('log N')
plt.ylabel('w mean std')
plt.grid(axis='y', linestyle='--', color='grey')
plt.savefig('w_mean_std_sub')

# Plot 3: H mean vs log N
plt.figure(23)
x = range(num_powers) + np.ones(num_powers)
for i in range(sub_num_trials):
    plt.scatter(x, H_mean_sub[i,:], color='black')
plt.xlabel('log N')
plt.ylabel('H mean (bits)')
plt.grid(axis='y', linestyle='--', color='grey')
plt.savefig('H_mean_sub')

# Plot 4: H mean std vs log N
plt.figure(24)
x = range(num_powers) + np.ones(num_powers)
for i in range(sub_num_trials):
    plt.scatter(x, H_mean_std_sub[i,:], color='black')
plt.xlabel('log N')
plt.ylabel('H mean std (bits)')
plt.grid(axis='y', linestyle='--', color='grey')
plt.savefig('H_mean_std_sub')

print ''
print '--- done ---'
print

In [ ]:
import pandas as pd

powers = ['1', '2', '3', '4', '5', '6']
columns = []
for i in range(num_powers):
    columns.append(powers[i])
    
df_w_mean_sub = pd.DataFrame(data=w_mean_sub, columns=columns)
df_w_mean_sub.index.name = '# trial'
df_w_mean_sub.to_csv('w_mean_sub.txt', sep='\t')

df_w_mean_std_sub = pd.DataFrame(data=w_mean_std_sub, columns=columns)
df_w_mean_std_sub.index.name = '# trial'
df_w_mean_std_sub.to_csv('w_mean_std_sub.txt', sep='\t')

df_H_mean_sub = pd.DataFrame(data=H_mean_sub, columns=columns)
df_H_mean_sub.index.name = '# trial'
df_H_mean_sub.to_csv('H_mean_sub.txt', sep='\t')

df_H_mean_std_sub = pd.DataFrame(data=H_mean_std_sub, columns=columns)
df_H_mean_std_sub.index.name = '# trial'
df_H_mean_std_sub.to_csv('H_mean_std_sub.txt', sep='\t')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.ion()

sns.set(style='whitegrid')

plt.figure(5)
fig_w_mean = sns.violinplot(data=df_w_mean_sub, color='white')
fig_w_mean.set_xlabel('log_N')
fig_w_mean.set_ylabel('w mean')
fig_w_mean.set_ylim(0.2,2.2)
plt.savefig('w_mean_vp')

plt.figure(6)
fig_w_mean_std = sns.violinplot(data=df_w_mean_std_sub, color='white')
fig_w_mean_std.set_xlabel('log_N')
fig_w_mean_std.set_ylabel('w mean std')
fig_w_mean_std.set_ylim(0.0,0.1)
plt.savefig('w_mean_std_vp')

plt.figure(7)
fig_H_mean = sns.violinplot(data=df_H_mean_sub, color='white', inner='quart')
fig_H_mean.set_xlabel('log_N')
fig_H_mean.set_ylabel('H mean (bits)')
plt.savefig('H_mean_vp')

plt.figure(8)
fig_H_mean_std = sns.violinplot(data=df_H_mean_std_sub, color='white')
fig_H_mean_std.set_xlabel('log_N')
fig_H_mean_std.set_ylabel('H mean std (bits)')
fig_H_mean_std.set_ylim(0.0,0.01)
plt.savefig('H_mean_std_vp')

In [ ]:
id_max = w_mean_sub[:,4].argmax()
data_seed = data_seeds[indices[id_max]]

# Without correction to Z
print 'Without correction to Z ...'
results = TestCase(N=50,
                   data_seed=data_seed,
                   G=100,
                   alpha=3,
                   bbox=[-20,20],
                   Z_eval='GLap',
                   num_Z_samples=0,
                   DT_MAX=1.0,
                   pt_method=None,
                   num_pt_samples=0,
                   fix_t_at_t_star=True
                   ).run()
results_naive = results
ts = [p.t for p in results.results.points]
log_Es = [p.log_E for p in results.results.points]
print ''

# With correcton to Z
print 'With correction to Z ...'
results = TestCase(N=50,
                   data_seed=data_seed,
                   G=100,
                   alpha=3,
                   bbox=[-20,20],
                   Z_eval='GLap+Sam',
                   num_Z_samples=10**5,
                   DT_MAX=1.0,
                   pt_method=None,
                   num_pt_samples=0,
                   fix_t_at_t_star=True
                   ).run()
results_weighted = results
ts_corr = [p.t for p in results.results.points]
log_Es_corr = [p.log_E for p in results.results.points]
print ''

plt.figure(9)
plt.plot(ts, log_Es, color='grey', alpha=0.5, zorder=1)
plt.scatter(ts, log_Es, color='grey', alpha=0.5, label='w/o correction', zorder=2)
plt.scatter(ts_corr, log_Es_corr, color='red', label='w/ correction', zorder=3)
plt.xlabel('t')
plt.ylabel('log_E')
plt.title('Case: max correction to Z')
plt.legend()
plt.savefig('max_corr')
plt.show()

In [ ]:
xs = results_naive.results.bin_centers
R = results_naive.results.R
h = results_naive.results.h
Q_true = results_naive.Q_true
Q_star_naive = results_naive.results.Q_star
Q_star_weighted = results_weighted.results.Q_star
plt.bar(xs, R, width=h, color='grey', alpha=0.5)
plt.plot(xs, Q_true, color='black')
plt.plot(xs, Q_star_naive, color='green')
plt.plot(xs, Q_star_weighted, color='red')
plt.show()

In [ ]:
id_min = w_mean_sub[:,4].argmin()
data_seed = data_seeds[indices[id_min]]

# Without correction to Z
print 'Without correction to Z ...'
results = TestCase(N=50,
                   data_seed=data_seed,
                   G=100,
                   alpha=3,
                   bbox=[-20,20],
                   Z_eval='GLap',
                   num_Z_samples=0,
                   DT_MAX=1.0,
                   pt_method=None,
                   num_pt_samples=0,
                   fix_t_at_t_star=True
                   ).run()
results_naive = results
ts = [p.t for p in results.results.points]
log_Es = [p.log_E for p in results.results.points]
print ''

# With correction to Z
print 'With correction to Z ...'
results = TestCase(N=50,
                   data_seed=data_seed,
                   G=100,
                   alpha=3,
                   bbox=[-20,20],
                   Z_eval='GLap+Sam',
                   num_Z_samples=10**5,
                   DT_MAX=1.0,
                   pt_method=None,
                   num_pt_samples=0,
                   fix_t_at_t_star=True
                   ).run()
results_weighted = results
ts_corr = [p.t for p in results.results.points]
log_Es_corr = [p.log_E for p in results.results.points]
print ''

plt.figure(10)
plt.plot(ts, log_Es, color='grey', alpha=0.5, zorder=1)
plt.scatter(ts, log_Es, color='grey', alpha=0.5, label='w/o correction', zorder=2)
plt.scatter(ts_corr, log_Es_corr, color='red', label='w/ correction', zorder=3)
plt.xlabel('t')
plt.ylabel('log_E')
plt.title('Case: min correction to Z')
plt.legend()
plt.savefig('min_corr')
plt.show()

In [ ]:
xs = results_naive.results.bin_centers
R = results_naive.results.R
h = results_naive.results.h
Q_true = results_naive.Q_true
Q_star_naive = results_naive.results.Q_star
Q_star_weighted = results_weighted.results.Q_star
plt.bar(xs, R, width=h, color='grey', alpha=0.5)
plt.plot(xs, Q_true, color='black')
plt.plot(xs, Q_star_naive, color='green')
plt.plot(xs, Q_star_weighted, color='red')
plt.show()